In [ ]:
# Notebook: Análisis Exploratorio de Ventas

# Paso 1: Importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Paso 2: Cargar los datos desde el archivo CSV
ruta_archivo = '../data/retail_sales_analysis.csv'
datos = np.genfromtxt(ruta_archivo, delimiter=',', dtype=str, skip_header=1)



In [ ]:
# Paso 3: Definir los encabezados manualmente
headers = ['Transaction ID', 'Date', 'Customer ID', 'Gender', 'Age', 'Product Category', 'Quantity', 'Price per Unit', 'Total Amount']

# Convertir a DataFrame
df = pd.DataFrame(datos, columns=headers)

# Convertir tipos de datos
df['Total Amount'] = pd.to_numeric(df['Total Amount'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Price per Unit'] = pd.to_numeric(df['Price per Unit'], errors='coerce')
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna()




In [ ]:
# Paso 4: Transformación de Datos
# Crear columna de ingreso total por venta (redundante en este caso, pero ilustrativo)
df['Calculated Total'] = df['Quantity'] * df['Price per Unit']

# Normalizar las ventas (Total Amount)
df['Normalized Total'] = (df['Total Amount'] - df['Total Amount'].mean()) / df['Total Amount'].std()

# Clasificar ventas en categorías
conditions = [
    df['Total Amount'] >= df['Total Amount'].quantile(0.75),
    df['Total Amount'] >= df['Total Amount'].quantile(0.25),
    df['Total Amount'] < df['Total Amount'].quantile(0.25)
]
choices = ['Alta', 'Media', 'Baja']
df['Venta Categoria'] = np.select(conditions, choices, default='Desconocido')



In [ ]:
# Paso 5: Agrupación y Agregación
# Agrupar por producto y fecha (mes)
df['Mes'] = df['Date'].dt.to_period('M')
grupo_producto_mes = df.groupby(['Product Category', 'Mes'])['Total Amount'].agg(['sum', 'mean', 'count', 'min', 'max', 'std', 'var'])



In [ ]:
# Paso 6: Análisis Personalizado con apply
# Desviación de cada venta respecto a la media de su grupo por categoría
media_por_categoria = df.groupby('Product Category')['Total Amount'].transform('mean')
df['Desviacion de la Media'] = df['Total Amount'] - media_por_categoria

# Mostrar resultados
print("Primeras filas del DataFrame transformado:")
print(df.head())
print("\nAgrupación por categoría y mes:")
print(grupo_producto_mes.head())